In [13]:
import matplotlib
from pathlib import Path
import mne
import mne_bids
from mne_bids import BIDSPath, write_raw_bids

matplotlib.use('Qt5Agg')

# Read the input data we wish to convert to BIDS

In [14]:
sample_data_dir = mne.datasets.sample.data_path()

# Convert to a pathlib.Path for more convenience
sample_data_dir = Path(sample_data_dir)
raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)
raw

Opening raw data file C:\Users\User\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


<Raw | sample_audvis_raw.fif, 376 x 166800 (277.7 s), ~3.2 MiB, data not loaded>

In [15]:
events = mne.find_events(raw)
event_id = {
    "Auditory/Left": 1,
    "Auditory/Right": 2,
    "Visual/Left": 3,
    "Visual/Right": 4,
    "Smiley": 5,
    "Button": 32
}
event_id

Finding events on: STI 014
320 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]


{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Visual/Left': 3,
 'Visual/Right': 4,
 'Smiley': 5,
 'Button': 32}

# Write the raw data to BIDS!

In [16]:
raw.info['line_freq']

Now actually write the data.

In [29]:
out_path = Path('out_data/sample_BIDS')

bids_path = BIDSPath(subject='02',
                     session='01',
                     task='audiovisual',
                     run='01',
                     root=out_path)

# Write raw data to BIDS format
write_raw_bids(raw, bids_path=bids_path, events=events,
               event_id=event_id, overwrite=True)

Opening raw data file C:\Users\User\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Writing 'out_data\sample_BIDS\participants.tsv'...
Writing 'out_data\sample_BIDS\participants.json'...
Writing 'out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_coordsystem.json'...
Writing 'out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_coordsystem.json'...
Used Annotations descriptions: [np.str_('Auditory/Left'), np.str_('Auditory/Right'), np.str_('Button'), np.str_('Smiley'), np.str_('Visual/Left'), np.str_('Visual/Right')]
Writing 'out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_task-audiovisual_run-01_events.tsv'...
Writing 'out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_task-audiovisual_run-01_events.json'...
Writing 'out_data\sample_BIDS\dataset_description.json'...

BIDSPath(
root: out_data/sample_BIDS
datatype: meg
basename: sub-02_ses-01_task-audiovisual_run-01_meg.fif)

# Write MEGIN/Elekta/NeuroMag fine-calibration and crosstalk files

In [30]:
cal_path=sample_data_dir/'SSS'/'sss_cal_mgh.dat'
ct_path=sample_data_dir/'SSS'/'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path,bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path,bids_path=bids_path)

Writing fine-calibration file to out_data/sample_BIDS/sub-02/ses-01/meg/sub-02_ses-01_acq-calibration_meg.dat
Writing crosstalk file to out_data/sample_BIDS/sub-02/ses-01/meg/sub-02_ses-01_acq-crosstalk_meg.fif


## Print the created file & directory structure

In [31]:
mne_bids.print_dir_tree(out_path)

|sample_BIDS\
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01\
|------ ses-01\
|--------- sub-01_ses-01_scans.tsv
|--------- meg\
|------------ sub-01_ses-01_acq-calibration_meg.dat
|------------ sub-01_ses-01_acq-crosstalk_meg.fif
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_events.json
|------------ sub-01_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.fif
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.json
|--- sub-02\
|------ ses-01\
|--------- sub-02_ses-01_scans.tsv
|--------- meg\
|------------ sub-02_ses-01_acq-calibration_meg.dat
|------------ sub-02_ses-01_acq-crosstalk_meg.fif
|------------ sub-02_ses-01_coordsystem.json
|------------ sub-02_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_events.json
|--

## Generate a data summary

In [32]:
print(mne_bids.make_report(out_path))

Summarizing participants.tsv out_data\sample_BIDS\participants.tsv...
Summarizing scans.tsv files [WindowsPath('out_data/sample_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv'), WindowsPath('out_data/sample_BIDS/sub-02/ses-01/sub-02_ses-01_scans.tsv')]...
The participant template found: sex were all unknown;
handedness were all unknown;
ages all unknown
 The [Unspecified] dataset was created by [Unspecified1], and [Unspecified2] and
conforms to BIDS version 1.7.0. This report was generated with MNE-BIDS
(https://doi.org/10.21105/joss.01896). The dataset consists of 2 participants
(sex were all unknown; handedness were all unknown; ages all unknown) and 1
recording sessions: 01. Data was recorded using an MEG system (Elekta) sampled
at 600.61 Hz with line noise at n/a Hz. The following software filters were
applied during recording: SpatialCompensation. There were 2 scans in total.
Recording durations ranged from 277.71 to 277.71 seconds (mean = 277.71, std =
0.0), for a total of 555.43 sec

## Reading BIDS data